In [1]:
import re 

In [2]:
class TSP:
    def __read (file):
        f = open(file, 'r')
        match = '^[0-9]\number.*'
        points = []
        
        for line in f:
            is_point = re.search(match, line)
            
            if is_point:
                points += [float(i) for i in line.split()][1:]
    
        return(points)
    
    def __init__(self, file):
        self.points = read(file)

In [ ]:
    def __read (file):
        f = open(file, 'r')
        match = '^[0-9]\number.*'
        points = []
        
        for line in f:
            is_point = re.search(match, line)
            
            if is_point:
                points += [float(i) for i in line.split()][1:]
    
        return(points)